In [4]:
import pandas as pd
import numpy as np
import datetime
from tqdm import tqdm

from geopy.distance import geodesic

%matplotlib inline
import mplleaflet
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.pylab as pylab
import seaborn as sns

import os
import warnings

from k_medoids import KMedoidsWeighted
import helper as hp

warnings.simplefilter("ignore")
tqdm.pandas()

np.random.seed(3)

In [5]:
data_dir = '../data/' # Diretório dos arquivos
quimio_name = 'Pulmão Quimioterapia SIA-SUS.csv' # Tipo de câncer a ser analisado (quimioterapia)
radio_name = 'Pulmão Radioterapia SIA-SUS.csv' # Tipo de câncer a ser analisado (radioterapia)
municipios_name = 'dados_municipios.csv'  #Dados dos municípios
area_name = 'dados_area.csv' #Área dos estados

dtype_cols = {'AP_MUNPCN': str, 'AP_UFMUN': str}
usecols = ['AP_MUNPCN', 'AP_UFMUN']

In [33]:
# Lendo os dataframes:

quimio = pd.read_csv(os.path.join(data_dir, quimio_name), encoding='latin', 
                     error_bad_lines=False, low_memory=False, dtype=dtype_cols, usecols=usecols)

radio = pd.read_csv(os.path.join(data_dir, radio_name), encoding='latin',
                    error_bad_lines=False, low_memory=False, dtype=dtype_cols, usecols=usecols)

municipios = pd.read_csv(os.path.join(data_dir, municipios_name), index_col=0, dtype={'COD_MUN': str})

area =  pd.read_csv(os.path.join(data_dir, area_name), encoding='latin', sep = ';', 
                    index_col=0, dtype={'COD': str})

In [34]:
# Removendo o último digito para se adequar ao dataframe das APACs:

municipios.index = municipios['COD_MUN'].apply(lambda x: x[:-1])

In [35]:
cancer = pd.concat([quimio, radio])

In [36]:
cancer['DIST'] = cancer.progress_apply(lambda row: hp.calculte_distance(row, municipios), axis=1)

100%|██████████| 330771/330771 [01:09<00:00, 4763.10it/s]


In [69]:
#Criando um dicionario entre os estados e os códigos:

lista_codigo_uf = municipios['CODIGO_UF'].unique().astype(str) 
lista_nome_uf = municipios['UF'].unique().astype(str)

#Remover 'DF'
lista_nome_uf = np.delete(lista_nome_uf, np.where(lista_uf == 'DF'))
nome_to_codigo_uf = dict(zip(lista_nome_uf, lista_codigo_uf))
codigo_to_nome_uf = dict(zip(lista_codigo_uf, lista_nome_uf))

In [65]:
locais_de_tratamento = {}
distancias = {}
indicadores = {}
for uf in lista_uf:
    codigo_uf = str(nome_to_codigo_uf[uf])
    cancer_estado = hp.generate_df_estado(cancer, municipios, codigo_uf)
    locais_de_tratamento[codigo_uf] = len(cancer_estado['AP_UFMUN'].unique())
    distancias[codigo_uf] = df_estado[df_estado['DIST']>0]['DIST'].mean()
    sqrtA = np.sqrt(area.loc[int(codigo_uf)]['ÁREA'])
    indicadores[codigo_uf] = distancias[codigo_uf]/sqrtA

In [72]:
df = pd.DataFrame({'Número de centros de tratamento': locais_de_tratamento, 
     'Distância média': distancias, 
     'Índice': indicadores, 'Nome': codigo_to_nome_uf})
df.sort_values(by=['Índice'], inplace = True)

In [73]:
df

,Número de centros de tratamento,Distância média,Índice,Nome
13,1.0,160.283849,0.128364,AM
15,2.0,160.283849,0.143480,PA
51,3.0,160.283849,0.168639,MT
31,24.0,160.283849,0.209290,MG
29,7.0,160.283849,0.213289,BA
50,4.0,160.283849,0.268205,MS
52,2.0,160.283849,0.274840,GO
21,2.0,160.283849,0.278203,MA
43,21.0,160.283849,0.301977,RS
17,2.0,160.283849,0.304149,TO
